# 📊 Repositório de Consultas SQL para Analistas de Dados: **Departamento Data Insights**
-  🔍 Contribuições: Estamos abertos a contribuições da comunidade de analistas de dados! Se você tiver consultas SQL úteis que gostaria de compartilhar ou sugestões para melhorias, sinta-se à vontade para enviar um pull request. Juntos, podemos expandir e aprimorar este repositório para beneficiar toda a comunidade de análise de dados.
- 🎯 Sinta-se à vontade para explorar, utilizar e contribuir para este repositório, e que ele possa ser uma ferramenta valiosa em sua jornada como analista de dados!

# **Market Basket Analysis - Crossbasket (Análise de Cesta de Compras)**
- A Market Basket Analysis, ou Análise de Cesta de Compras, especialmente no contexto de "Crossbasket", é uma técnica estatística utilizada para entender as relações entre diferentes produtos comprados pelos clientes durante suas compras. Esta análise examina padrões e tendências de compra, determinando quais itens são frequentemente comprados juntos.

### **Explicação da Query:**
- Paired_Products: Esta CTE cria pares de produtos que foram comprados juntos no mesmo pedido. A condição oi1.product_id < oi2.product_id garante que cada combinação seja contada apenas uma vez e evita duplicidade.
- Product_Combinations: Agrupa esses pares para contar quantas vezes cada par de produtos foi pedido.
- Total_Orders_Per_Product: Conta quantos pedidos contêm cada produto individualmente.
- Consulta Final: Calcula a porcentagem de pedidos contendo o produto menos frequente no par que também inclui o outro produto do par. Isso é feito dividindo o número de pedidos que contêm ambos os produtos pela quantidade total de pedidos do produto menos comum no par. Os resultados são ordenados por order_count para destacar os pares mais comuns.

💡 Essa análise ajuda a identificar oportunidades para promoções cruzadas ou recomendações de produtos baseadas em padrões de compra observados, aumentando potencialmente as vendas e a satisfação do cliente ao antecipar suas necessidades.

### **Cálculo da Porcentagem:**
Na consulta SQL que preparamos, a porcentagem é calculada da seguinte forma:

Primeiro, identificamos todos os pedidos que contêm cada par possível de produtos (Produto1 e Produto2) que foram comprados juntos.
Em seguida, contamos o número total de pedidos que contêm cada um desses produtos individualmente.
A porcentagem é calculada usando o número de pedidos que contêm ambos os produtos dividido pelo número total de pedidos que contêm o produto menos comum no par. Isso é feito para destacar a força da associação entre os dois produtos.

### **Explicação da "Porcentagem":**
"Produto menos comum no par": Este é o produto que aparece em menos pedidos entre os dois produtos que estamos comparando. Por exemplo, se o Produto A está em 100 pedidos e o Produto B em 150, então o Produto A é o "produto menos comum".
Cálculo da porcentagem: A porcentagem mostra qual fração dos pedidos que contêm o produto menos comum também contêm o outro produto. Se a porcentagem é alta, isso indica uma forte associação entre os dois produtos no contexto de compras conjuntas.

### **Exemplo Prático**
*Imagine que:*

- Produto A está em 100 pedidos.
- Produto B está em 150 pedidos.
- Ambos, Produto A e B, estão juntos em 50 pedidos.

*Neste caso:*

Produto A é o menos comum (100 pedidos vs. 150 pedidos de Produto B).
A porcentagem será (50 / 100) * 100 = 50%.
Isso significa que, dos pedidos que incluem Produto A, 50% também incluem Produto B. Esta é uma métrica crucial para entender quão dependente é a compra do Produto B quando o Produto A é comprado, indicando uma possível recomendação de Produto B para os compradores de Produto A.

💡 Essa análise ajuda a identificar oportunidades para promoções cruzadas ou recomendações de produtos baseadas em padrões de compra observados, aumentando potencialmente as vendas e a satisfação do cliente ao antecipar suas necessidades.

In [ ]:
%%sql
WITH Paired_Products AS (
    SELECT
        oi1.order_id,
        oi1.product_id AS product_id1,
        oi2.product_id AS product_id2
    FROM
        `bigquery-public-data.thelook_ecommerce.order_items` oi1
    JOIN
        `bigquery-public-data.thelook_ecommerce.order_items` oi2
        ON oi1.order_id = oi2.order_id AND oi1.product_id < oi2.product_id
    WHERE
        oi1.status = 'Complete' AND oi2.status = 'Complete'
),
Product_Combinations AS (
    SELECT
        product_id1,
        product_id2,
        COUNT(DISTINCT order_id) AS order_count
    FROM
        Paired_Products
    GROUP BY
        product_id1, product_id2
),
Total_Orders_Per_Product AS (
    SELECT
        product_id,
        COUNT(DISTINCT order_id) AS total_orders
    FROM
        `bigquery-public-data.thelook_ecommerce.order_items`
    WHERE
        status = 'Complete'
    GROUP BY
        product_id
)

SELECT
    pc.product_id1,
    pc.product_id2,
    pc.order_count,
    ROUND((CAST(pc.order_count AS FLOAT64) / MIN(tp.total_orders)) * 100, 2) AS percentage
FROM
    Product_Combinations pc
JOIN
    Total_Orders_Per_Product tp ON pc.product_id1 = tp.product_id OR pc.product_id2 = tp.product_id
GROUP BY
    pc.product_id1, pc.product_id2, pc.order_count
ORDER BY
    pc.order_count DESC;

# **Lifetime Value (LTV) dos Clientes**
- O Lifetime Value (LTV) dos Clientes é uma métrica financeira que estima o valor total de receita ou lucro que um cliente contribui para uma empresa ao longo de todo o seu relacionamento com ela. O LTV calcula não apenas as vendas imediatas, mas também incorpora a projeção de todas as futuras transações que o cliente pode realizar. Esta métrica é crucial para empresas entenderem o valor a longo prazo de manter relacionamentos com clientes, auxiliando na decisão de quanto investir em aquisição e retenção de clientes. Um LTV alto indica um cliente altamente valioso, que justifica investimentos maiores em marketing e serviços para garantir sua satisfação e fidelidade.

In [ ]:
%%sql
SELECT
    cliente_id,
    AVG(valor_total) AS ltv
FROM pedidos
GROUP BY cliente_id;

# **Custo de Aquisição de Clientes (CAC)**
- O Custo de Aquisição de Clientes (CAC) é uma métrica financeira que calcula o custo médio envolvido na aquisição de um novo cliente. Ele inclui todos os custos de marketing e vendas divididos pelo número de clientes adquiridos durante um período específico. O CAC é essencial para avaliar a eficácia das estratégias de marketing e vendas de uma empresa, ajudando a entender quanto está sendo efetivamente investido para ganhar cada novo cliente. Uma gestão eficiente do CAC é crucial para garantir que a empresa não esteja gastando excessivamente para atrair clientes, mantendo a rentabilidade.

In [ ]:
%%sql
SELECT
    DATE_TRUNC('month', data_cadastro) AS mes,
    SUM(custo_campanhas + custo_vendas) / COUNT(DISTINCT cliente_id) AS cac_mensal
FROM clientes
GROUP BY mes
ORDER BY mes;

# **Taxa de Engajamento nas Redes Sociais**
- A Taxa de Engajamento nas Redes Sociais é uma métrica que mede a interação dos usuários com o conteúdo publicado em plataformas de mídia social. Ela é calculada com base no número de interações ativas que o conteúdo recebe, como curtidas, comentários, compartilhamentos e visualizações, dividido pelo número total de seguidores ou alcance do post. Essa taxa é um indicador importante da relevância e do impacto do conteúdo, mostrando o quanto ele é capaz de envolver e atrair a atenção dos usuários.

In [ ]:
%%sql
SELECT (Interações / Seguidores) AS Engajamento
FROM redes_sociais
WHERE plataforma = 'Instagram'
AND data_postagem BETWEEN '2024-01-01' AND '2024-03-31';

# **Tempo Médio de Resposta (MTTR)**
- Tempo Médio de Resposta (MTTR) é uma métrica que indica o tempo médio que leva para resolver problemas técnicos ou responder a solicitações dos clientes após eles serem identificados. É essencial para avaliar a eficiência de equipes de suporte e manutenção.

In [ ]:
%%sql
SELECT AVG(TIMESTAMPDIFF(MINUTE, created_at, resolved_at)) AS tempo_medio_resolucao
FROM tickets_suporte
WHERE resolved_at IS NOT NULL AND data BETWEEN '2024-04-01' AND '2024-04-30';

# **Taxa de Retenção de Usuários**
- A Taxa de Retenção de Clientes é uma métrica que mede a porcentagem de clientes que retornam para fazer pedidos novamente ou renovam suas assinaturas em um determinado período. Essencialmente, ela reflete o sucesso de uma empresa em manter seus clientes engajados e satisfeitos ao longo do tempo. Uma alta taxa de retenção é geralmente um indicador de boa saúde do cliente e da eficácia das estratégias de relacionamento e fidelização da empresa.

📋 **Explicação da Query:**

- pedidos_mensais é um CTE (Common Table Expression) que agrupa pedidos por usuário e mês/ano, para identificar em quais meses os usuários estiveram ativos.
- clientes_retidos é outro CTE que junta os dados do mesmo usuário entre dois meses consecutivos, verificando se o cliente que fez um pedido em um mês também fez no seguinte.
- A query final seleciona o ano e mês de início, conta o total de clientes únicos e quantos desses foram retidos no mês seguinte.
- Calcula a taxa de retenção como a proporção de clientes retidos em relação ao total de clientes do mês anterior.

Ajuste as colunas e a lógica conforme necessário para se alinhar com a estrutura e as necessidades específicas de seus dados.

In [ ]:
%%sql
-- Suponha que temos uma tabela chamada 'pedidos' com colunas 'id_usuario' e 'data_pedido'
WITH pedidos_mensais AS (
    SELECT
        id_usuario,
        EXTRACT(YEAR FROM data_pedido) AS ano,
        EXTRACT(MONTH FROM data_pedido) AS mes
    FROM
        pedidos
    GROUP BY
        id_usuario, ano, mes
),

clientes_retidos AS (
    SELECT
        a.id_usuario,
        a.ano AS ano_inicio,
        a.mes AS mes_inicio,
        b.ano AS ano_retido,
        b.mes AS mes_retido
    FROM
        pedidos_mensais a
    JOIN
        pedidos_mensais b ON a.id_usuario = b.id_usuario
    WHERE
        (b.ano > a.ano OR (b.ano = a.ano AND b.mes = a.mes + 1))
)

SELECT
    ano_inicio,
    mes_inicio,
    COUNT(DISTINCT id_usuario) AS total_clientes,
    COUNT(DISTINCT id_usuario) FILTER (WHERE ano_retido IS NOT NULL AND mes_retido IS NOT NULL) AS clientes_retidos,
    ROUND((COUNT(DISTINCT id_usuario) FILTER (WHERE ano_retido IS NOT NULL AND mes_retido IS NOT NULL) * 100.0) / COUNT(DISTINCT id_usuario), 2) AS taxa_retencao
FROM
    clientes_retidos
GROUP BY
    ano_inicio, mes_inicio
ORDER BY
    ano_inicio, mes_inicio;

# **Análise de Cohort**
- Agrupa os clientes com base em características semelhantes para analisar seu comportamento ao longo do tempo.

In [ ]:
%%sql
-- Em Desenvolvimento

# **Taxa de Churn**
- Mede a proporção de clientes que cancelam ou não renovam seus serviços em um determinado período de tempo.

In [ ]:
%%sql
-- Em Desenvolvimento